# Report mAIstro Notebook

This notebook demonstrates how to use the Report mAIstro tool to generate customizable reports on any user-supplied topic.

In [1]:
# Import necessary libraries
import os
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph
from langgraph.constants import Send
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from tavily import TavilyClient, AsyncTavilyClient
from pydantic import BaseModel, Field
from typing import List
from typing_extensions import TypedDict
import asyncio
import operator
import configuration
from langsmith import traceable
from langchain_core.messages import HumanMessage, SystemMessage


## Step 1: Set up the environment

Make sure you have the necessary API keys in your `.env` file.

In [2]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

## Step 2: Define the configuration

Define the configuration for the report generation process.

In [3]:
# Define the configuration
config = RunnableConfig(
    configurable={
        "report_structure": configuration.DEFAULT_REPORT_STRUCTURE,
        "number_of_queries": 2,
        "tavily_topic": "general",
        "tavily_days": None
    }
)

## Step 3: Define the report state

Define the initial state for the report generation process.

In [4]:
# Define the initial state
initial_state = {
    "topic": "The impact of AI on modern business",
    "sections": [],
    "completed_sections": [],
    "report_sections_from_research": "",
    "final_report": ""
}

## Step 4: Define the report generation graph

Define the state graph for the report generation process.

In [5]:
# Define the report generation graph
graph = StateGraph(
    input=initial_state,
    output={"final_report": str},
    config=config
)

# Add nodes and edges to the graph
graph.add_node("generate_report_plan", generate_report_plan)
graph.add_node("build_section_with_web_research", section_builder.compile())
graph.add_node("gather_completed_sections", gather_completed_sections)
graph.add_node("write_final_sections", write_final_sections)
graph.add_node("compile_final_report", compile_final_report)
graph.add_edge(START, "generate_report_plan")
graph.add_conditional_edges("generate_report_plan", initiate_section_writing, ["build_section_with_web_research"])
graph.add_edge("build_section_with_web_research", "gather_completed_sections")
graph.add_conditional_edges("gather_completed_sections", initiate_final_section_writing, ["write_final_sections"])
graph.add_edge("write_final_sections", "compile_final_report")
graph.add_edge("compile_final_report", END)

# Compile the graph
compiled_graph = graph.compile()

## Step 5: Run the report generation process

Run the state graph to generate the report.

In [6]:
# Run the report generation process
final_report = compiled_graph.run()
print(final_report)